In [1]:
from toy_classify import *
import ot
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

/users/1/tianx/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## Adasyn

In [2]:
from imblearn.over_sampling import ADASYN

In [3]:
list_result=[]
ce_smote=[]
ce_origin=[]
list_origin=[]

num_sim=10
num_split= len(ratio_list )

# Set data sizes.
n_minority = 800
n_majority = 3200
n_val = 400
n_test = 800
n_train =4000

#hyperparameter sets
ratio_list = [x/10 for x in list(range(1,11))] #split ratio r
candidate_m_syn=  [int(n_train*round(x * 0.1, 2)) for x in range(21)] #synthetic sample size m/n
candidate_alpha_scale= [0.1, 0.2, 0.3,0.4, 0.5,0.6, 0.7,0.8, 0.9] #alpha


W_list=[[] for j in range(10)]
X_total, y_total, regions_total = generate_imbalanced_data(
        n_minority + n_val + n_test, 
        n_majority + n_val + n_test, 
        seed=0
    )
y_total = y_total.reshape(-1, 1)    
for j in range(10):
    set_seed(j)
    
    # Separate the minority and majority groups based on region labels.
    # Here, we assume that region label 0 indicates minority and 1 indicates majority.
    minority_idx = np.where(regions_total == 0)[0]
    majority_idx = np.where(regions_total == 1)[0]

    # Shuffle indices separately
    np.random.shuffle(minority_idx)
    np.random.shuffle(majority_idx)

    # For minority:
    min_train_idx = minority_idx[:n_minority]
    min_val_idx   = minority_idx[n_minority : n_minority + n_val]
    min_test_idx  = minority_idx[n_minority + n_val : n_minority + n_val + n_test]

    # For majority:
    maj_train_idx = majority_idx[:n_majority]
    maj_val_idx   = majority_idx[n_majority : n_majority + n_val]
    maj_test_idx  = majority_idx[n_majority + n_val : n_majority + n_val + n_test]

    # Combine directly for each set
    X_train_orig = np.vstack((X_total[min_train_idx], X_total[maj_train_idx]))
    y_train_orig = np.vstack((y_total[min_train_idx], y_total[maj_train_idx]))
    regions_train = np.concatenate((regions_total[min_train_idx], regions_total[maj_train_idx]))

    X_val = np.vstack((X_total[min_val_idx], X_total[maj_val_idx]))
    y_val = np.vstack((y_total[min_val_idx], y_total[maj_val_idx]))
    regions_val = np.concatenate((regions_total[min_val_idx], regions_total[maj_val_idx]))

    X_test = np.vstack((X_total[min_test_idx], X_total[maj_test_idx]))
    y_test = np.vstack((y_total[min_test_idx], y_total[maj_test_idx]))
    regions_test = np.concatenate((regions_total[min_test_idx], regions_total[maj_test_idx]))

    
    XY_train_orig = combine_XY(X_train_orig, y_train_orig)

    origin_model, mse_val_origin, mse_test_origin, y_val_opred, y_test_opred = train_and_evaluate(
    X_train_orig, y_train_orig, X_val, y_val, X_test, y_test, input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    
    list_origin.append([mse_val_origin, mse_test_origin])

    all_result=[]

        
    for m_syn in candidate_m_syn:   
        # Generate synthetic data for minority (region 1)
        result=[]


        for alpha_scale in candidate_alpha_scale:
            # Adjust synthetic data: if alpha_scale < 1, use a fraction; if > 1, replicate data accordingly.
            if m_syn ==0:
                X_train_combined = XY_train_orig[:,:-1]
                y_train_combined = XY_train_orig[:,-1:]    
            else:
                
                resample_dict = {0: int(m_syn*alpha_scale)+ n_minority, 1: int(m_syn*(1-alpha_scale))+ n_majority}
                try:
                    over_sampler = ADASYN(sampling_strategy=resample_dict, random_state=0)
                    X_rs_df, y_rs = over_sampler.fit_resample(XY_train_orig[:, :-1], XY_train_orig[:, -1])
                except ValueError as e:
                    print(f"SMOTE also failed: {e}")
                    print("Falling back to original training data.")
                    X_rs_df = XY_train_orig[:, :-1]
                    y_rs = XY_train_orig[:, -1]

                X_train_combined = X_rs_df
                y_train_combined = y_rs


            # Train and evaluate on validation set
            _, mse_val, mse_test, _, _ = train_and_evaluate(
                X_train_combined, y_train_combined, X_val, y_val, X_test, y_test,
                input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)

            print(f"m_syn={m_syn}, alpha_scale={alpha_scale} -> Validation MSE: {mse_val:.4f} -> Test MSE: {mse_test:.4f}")

            result.append([mse_val, mse_test])

        all_result.append(result)
    
    ax= [all_result[x][:10] for x in range(21)] 
    ax=np.array(ax)
    num_sim, num_m, num_min_ratio = ax.shape

    val_errors = ax[: , :, 0]

    # Find the indices (k, l) that minimize the validation error.
    k, l = np.unravel_index(np.argmin(val_errors), val_errors.shape)

        
    best_m = candidate_m_syn[k]
    best_alpha = candidate_alpha_scale[l]
    
    resample_dict = {0: int(best_m*best_alpha)+ n_minority, 1: int(best_m*(1-best_alpha))+ n_majority}
    over_sampler = ADASYN(sampling_strategy=resample_dict, random_state=0)
    X_rs_df, y_rs = over_sampler.fit_resample(XY_train_orig[:, :-1], XY_train_orig[:, -1])



    X_train_combined = X_rs_df
    y_train_combined = y_rs
    
    ces_one={}


    _, mse_val, mse_test, ypred_val, ypred_test = train_and_evaluate(
        X_train_combined, y_train_combined, X_val, y_val, X_test, y_test,
        input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    ces_one["avg"]=mse_test
    
    _, mse_val, mse_test, ypred_val, ypred_test = train_and_evaluate(
    X_train_combined, y_train_combined, X_val, y_val, X_test[regions_test==1,], y_test[regions_test==1],
    input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    ces_one["major"]=mse_test
    
    _, mse_val, mse_test, ypred_val, ypred_test = train_and_evaluate(
    X_train_combined, y_train_combined, X_val, y_val, X_test[regions_test==0,], y_test[regions_test==0],
    input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    ces_one["minor"]=mse_test
    
    ce_smote.append(ces_one)
    
    ces_one={}
    _, mse_val_origin, mse_test_origin, y_val_opred, y_test_opred = train_and_evaluate(
    X_train_orig, y_train_orig, X_val, y_val, X_test, y_test, input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    ces_one["avg"]=mse_test_origin
    _, mse_val_origin, mse_test_origin, y_val_opred, y_test_opred = train_and_evaluate(
    X_train_orig, y_train_orig, X_val, y_val, X_test[regions_test==1,], y_test[regions_test==1], input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    ces_one["major"]=mse_test_origin
    _, mse_val_origin, mse_test_origin, y_val_opred, y_test_opred = train_and_evaluate(
    X_train_orig, y_train_orig, X_val, y_val, X_test[regions_test==0,], y_test[regions_test==0], input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    ces_one["minor"]=mse_test_origin
    
    ce_origin.append(ces_one)
    list_result.append(all_result)


/common/software/install/migrated/anaconda/python3-2020.07-mamba/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


Validation Accuracy: 0.73625
Test Accuracy: 0.701875
Validation BCE Metric: 0.48582834005355835
Test BCE Metric: 0.6221937537193298
Average Validation Entropy: nan
Average Test Entropy: nan
[0.48582834005355835, 0.6221937537193298]
Validation Accuracy: 0.73625
Test Accuracy: 0.701875
Validation BCE Metric: 0.48582834005355835
Test BCE Metric: 0.6221937537193298
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=0, alpha_scale=0.1 -> Validation MSE: 0.4858 -> Test MSE: 0.6222
Validation Accuracy: 0.73625
Test Accuracy: 0.701875
Validation BCE Metric: 0.48582834005355835
Test BCE Metric: 0.6221937537193298
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=0, alpha_scale=0.2 -> Validation MSE: 0.4858 -> Test MSE: 0.6222
Validation Accuracy: 0.73625
Test Accuracy: 0.701875
Validation BCE Metric: 0.48582834005355835
Test BCE Metric: 0.6221937537193298
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=0, alpha_scale=0.3 -> Validation MSE: 0.4858 -> Te

Validation Accuracy: 0.73625
Test Accuracy: 0.701875
Validation BCE Metric: 0.48582834005355835
Test BCE Metric: 0.6221937537193298
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=800, alpha_scale=0.9 -> Validation MSE: 0.4858 -> Test MSE: 0.6222
SMOTE also failed: No samples will be generated with the provided ratio settings.
Falling back to original training data.
Validation Accuracy: 0.73625
Test Accuracy: 0.701875
Validation BCE Metric: 0.48582834005355835
Test BCE Metric: 0.6221937537193298
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=1200, alpha_scale=0.1 -> Validation MSE: 0.4858 -> Test MSE: 0.6222
Early stopping at epoch 81
Validation Accuracy: 0.72
Test Accuracy: 0.695
Validation BCE Metric: 0.5616541504859924
Test BCE Metric: 0.6714462637901306
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=1200, alpha_scale=0.2 -> Validation MSE: 0.5617 -> Test MSE: 0.6714
Validation Accuracy: 0.745
Test Accuracy: 0.72125
Validation BCE Me

Early stopping at epoch 69
Validation Accuracy: 0.7025
Test Accuracy: 0.669375
Validation BCE Metric: 0.6294776201248169
Test BCE Metric: 0.7336837649345398
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=2400, alpha_scale=0.1 -> Validation MSE: 0.6295 -> Test MSE: 0.7337
Early stopping at epoch 95
Validation Accuracy: 0.72625
Test Accuracy: 0.700625
Validation BCE Metric: 0.49990150332450867
Test BCE Metric: 0.6175634860992432
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=2400, alpha_scale=0.2 -> Validation MSE: 0.4999 -> Test MSE: 0.6176
Early stopping at epoch 85
Validation Accuracy: 0.735
Test Accuracy: 0.708125
Validation BCE Metric: 0.47313201427459717
Test BCE Metric: 0.6067503690719604
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=2400, alpha_scale=0.3 -> Validation MSE: 0.4731 -> Test MSE: 0.6068
Validation Accuracy: 0.77625
Test Accuracy: 0.743125
Validation BCE Metric: 0.4176538288593292
Test BCE Metric: 0.5782177448272705


Early stopping at epoch 94
Validation Accuracy: 0.7425
Test Accuracy: 0.715625
Validation BCE Metric: 0.438302606344223
Test BCE Metric: 0.6110842227935791
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=3600, alpha_scale=0.3 -> Validation MSE: 0.4383 -> Test MSE: 0.6111
Early stopping at epoch 96
Validation Accuracy: 0.7775
Test Accuracy: 0.735625
Validation BCE Metric: 0.41931143403053284
Test BCE Metric: 0.6278302073478699
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=3600, alpha_scale=0.4 -> Validation MSE: 0.4193 -> Test MSE: 0.6278
Early stopping at epoch 84
Validation Accuracy: 0.8075
Test Accuracy: 0.77375
Validation BCE Metric: 0.38957417011260986
Test BCE Metric: 0.5408651828765869
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=3600, alpha_scale=0.5 -> Validation MSE: 0.3896 -> Test MSE: 0.5409
Early stopping at epoch 75
Validation Accuracy: 0.7925
Test Accuracy: 0.758125
Validation BCE Metric: 0.3957405686378479
Test BCE Met

...

Early stopping at epoch 59
Validation Accuracy: 0.74
Test Accuracy: 0.74375
Validation BCE Metric: 0.4848184585571289
Test BCE Metric: 0.4690234363079071
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=3200, alpha_scale=0.6 -> Validation MSE: 0.4848 -> Test MSE: 0.4690
Early stopping at epoch 53
Validation Accuracy: 0.76875
Test Accuracy: 0.744375
Validation BCE Metric: 0.4737389385700226
Test BCE Metric: 0.4640962481498718
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=3200, alpha_scale=0.7 -> Validation MSE: 0.4737 -> Test MSE: 0.4641
Early stopping at epoch 57
Validation Accuracy: 0.75875
Test Accuracy: 0.755
Validation BCE Metric: 0.47371676564216614
Test BCE Metric: 0.44963034987449646
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=3200, alpha_scale=0.8 -> Validation MSE: 0.4737 -> Test MSE: 0.4496
Early stopping at epoch 55
Validation Accuracy: 0.745
Test Accuracy: 0.761875
Validation BCE Metric: 0.47199684381484985
Test BCE Metri

Early stopping at epoch 54
Validation Accuracy: 0.735
Test Accuracy: 0.748125
Validation BCE Metric: 0.48314177989959717
Test BCE Metric: 0.45322707295417786
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=4400, alpha_scale=0.8 -> Validation MSE: 0.4831 -> Test MSE: 0.4532
Early stopping at epoch 50
Validation Accuracy: 0.74
Test Accuracy: 0.744375
Validation BCE Metric: 0.49216270446777344
Test BCE Metric: 0.46747350692749023
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=4400, alpha_scale=0.9 -> Validation MSE: 0.4922 -> Test MSE: 0.4675
Early stopping at epoch 89
Validation Accuracy: 0.71375
Test Accuracy: 0.71625
Validation BCE Metric: 0.6774677038192749
Test BCE Metric: 0.6346577405929565
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=4800, alpha_scale=0.1 -> Validation MSE: 0.6775 -> Test MSE: 0.6347
Early stopping at epoch 65
Validation Accuracy: 0.7075
Test Accuracy: 0.713125
Validation BCE Metric: 0.5866730213165283
Test BCE Me

Early stopping at epoch 87
Validation Accuracy: 0.70625
Test Accuracy: 0.70125
Validation BCE Metric: 0.6802954077720642
Test BCE Metric: 0.659404456615448
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=6000, alpha_scale=0.1 -> Validation MSE: 0.6803 -> Test MSE: 0.6594
Early stopping at epoch 54
Validation Accuracy: 0.71
Test Accuracy: 0.706875
Validation BCE Metric: 0.6286727786064148
Test BCE Metric: 0.5942374467849731
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=6000, alpha_scale=0.2 -> Validation MSE: 0.6287 -> Test MSE: 0.5942
Early stopping at epoch 67
Validation Accuracy: 0.72375
Test Accuracy: 0.714375
Validation BCE Metric: 0.550258219242096
Test BCE Metric: 0.5114139318466187
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=6000, alpha_scale=0.3 -> Validation MSE: 0.5503 -> Test MSE: 0.5114
Early stopping at epoch 58
Validation Accuracy: 0.725
Test Accuracy: 0.734375
Validation BCE Metric: 0.5257282257080078
Test BCE Metric:

Early stopping at epoch 59
Validation Accuracy: 0.71625
Test Accuracy: 0.719375
Validation BCE Metric: 0.5485486388206482
Test BCE Metric: 0.5018742084503174
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=7200, alpha_scale=0.3 -> Validation MSE: 0.5485 -> Test MSE: 0.5019
Early stopping at epoch 46
Validation Accuracy: 0.7075
Test Accuracy: 0.71375
Validation BCE Metric: 0.5432408452033997
Test BCE Metric: 0.515420138835907
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=7200, alpha_scale=0.4 -> Validation MSE: 0.5432 -> Test MSE: 0.5154
Early stopping at epoch 45
Validation Accuracy: 0.73
Test Accuracy: 0.736875
Validation BCE Metric: 0.4813252091407776
Test BCE Metric: 0.4628397226333618
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=7200, alpha_scale=0.5 -> Validation MSE: 0.4813 -> Test MSE: 0.4628
Early stopping at epoch 41
Validation Accuracy: 0.73875
Test Accuracy: 0.743125
Validation BCE Metric: 0.4812009334564209
Test BCE Metri

Early stopping at epoch 76
Validation Accuracy: 0.70375
Test Accuracy: 0.4425
Validation BCE Metric: 0.5902949571609497
Test BCE Metric: 0.9474532604217529
Average Validation Entropy: nan
Average Test Entropy: nan


### Summary statistics for Table 1

In [4]:
[np.mean([x['major'] for x in ce_smote]),np.mean([x['minor'] for x in ce_smote]),np.mean([x['avg'] for x in ce_smote])] 

[0.4387704163789749, 0.5140650987625122, 0.47641776204109193]

In [5]:
[np.std([x['major'] for x in ce_smote]),np.std([x['minor'] for x in ce_smote]),np.std([x['avg'] for x in ce_smote])] 

[0.04932962604278554, 0.09810987107326069, 0.034290652076275456]